<center><img width="25%" src="https://www.camaramirimdoce.sc.gov.br/media/noticia/resumo-da-sessao-10-06-2019-111.png?w=848&h=450&t=P&c=f0f0f0&q=80&v=2"></center>

---

<br>

# Detecção de notícias falsas

<br>

> https://github.com/deepgupta06/fake_news <br>
> https://www.youtube.com/watch?v=5X27excCyXk&ab_channel=yobots

In [ ]:
# Importando a biblioteca necessária
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
# Inserindo conjuntos de dados falsos e reais
df_falso = pd.read_csv("Fake.csv")
df_verdadeiro = pd.read_csv("True.csv")

In [ ]:
df_falso.head(5)

In [ ]:
df_verdadeiro.head(5)

In [ ]:
# Inserindo uma coluna chamada "classe" para o conjunto de dados de notícias falsas e reais nas categorias de notícias falsas e verdadeiras.
df_falso["class"] = 0
df_verdadeiro["class"] = 1

In [ ]:
# Removendo as últimas 10 linhas do conjunto de dados, para teste manual
df_falso.shape, df_verdadeiro.shape

In [ ]:
df_falso_manual_teste = df_falso.tail(10)
for i in range(23480,23470,-1):
    df_falso.drop([i], axis = 0, inplace = True)
    
df_verdadeiro_manual_teste = df_verdadeiro.tail(10)
for i in range(21416,21406,-1):
    df_verdadeiro.drop([i], axis = 0, inplace = True)

In [ ]:
df_falso.shape, df_verdadeiro.shape

In [ ]:
# Mesclar o dataframe de teste manual em um único conjunto de dados e salvá-lo em um arquivo csv
df_falso_manual_teste["class"] = 0
df_verdadeiro_manual_teste["class"] = 1

In [ ]:
df_falso_manual_teste.head(10)

In [ ]:
df_verdadeiro_manual_teste.head(10)

In [ ]:
df_manual_teste= pd.concat([df_falso_manual_teste,df_verdadeiro_manual_teste], axis = 0)
df_manual_teste.to_csv("manual_teste.csv")

In [ ]:
# Mesclando o principal dataframe falso e verdadeiro
df_juntos = pd.concat([df_falso, df_verdadeiro], axis =0 )
df_juntos.head(10)

In [ ]:
df_juntos.columns

In [ ]:
# As colunas "título", "assunto" e "data" não são necessárias para detectar notícias falsas, então vou eliminar as colunas.

df = df_juntos.drop(["title", "subject","date"], axis = 1)

In [ ]:
df.isnull().sum()

In [ ]:
# Embaralhando aleatoriamente o dataframe
df = df.sample(frac = 1)
df.head()

In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.columns

In [ ]:
df.head()

In [ ]:
# Criando uma função para converter o texto em minúsculas, remover o espaço extra, chr especial, ulr e links.
def converter(texto):
    texto = texto.lower()
    texto = re.sub('\[.*?\]', '', texto)
    texto = re.sub("\\W"," ",texto) 
    texto = re.sub('https?://\S+|www\.\S+', '', texto)
    texto = re.sub('<.*?>+', '', texto)
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto)
    texto = re.sub('\n', '', texto)
    texto = re.sub('\w*\d\w*', '', texto)    
    return texto

In [ ]:
df["text"] = df["text"].apply(converter)

In [ ]:
# Definindo variável dependente e independente como x e y
x = df["text"]
y = df["class"]

In [ ]:
# Dividindo o conjunto de dados em conjunto de treinamento e conjunto de teste.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
# Converter texto em vetores
from sklearn.feature_extraction.text import TfidfVectorizer

vetorizacao = TfidfVectorizer()
xv_treino = vetorizacao.fit_transform(x_train)
xv_teste = vetorizacao.transform(x_test)

In [ ]:
# 1. Regressão Logística
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()

LR.fit(xv_treino,y_train)

prever_lr=LR.predict(xv_teste)

LR.score(xv_teste, y_test)

In [ ]:
print(classification_report(y_test, prever_lr))

In [ ]:
# 2. Classificação da árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()

DT.fit(xv_treino, y_train)

prever_dt = DT.predict(xv_teste)

DT.score(xv_teste, y_test)

In [ ]:
print(classification_report(y_test, prever_dt))

In [ ]:
# 3. Classificador de aumento de gradiente

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)

GBC.fit(xv_treino, y_train)

prever_gbc = GBC.predict(xv_teste)

GBC.score(xv_teste, y_test)

In [ ]:
print(classification_report(y_test, prever_gbc))

In [ ]:
# 4. Classificador de floresta aleatório

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)

RFC.fit(xv_treino, y_train)

prever_rfc = RFC.predict(xv_teste)

RFC.score(xv_teste, y_test)

In [ ]:
print(classification_report(y_test, prever_rfc))

In [ ]:
# Teste de modelo com entrada manual

def rotulo_saida(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Não é uma notícia falsa"
    
def teste_manual(news):
    noticia_teste = {"text":[news]}
    df_definicao_novo_teste = pd.DataFrame(noticia_teste)
    df_definicao_novo_teste["text"] = df_definicao_novo_teste["text"].apply(converter) 
    novo_x_teste = df_definicao_novo_teste["text"]
    novo_xv_teste = vetorizacao.transform(novo_x_teste)
    prever_LR = LR.predict(novo_xv_teste)
    prever_DT = DT.predict(novo_xv_teste)
    prever_GBC = GBC.predict(novo_xv_teste)
    prever_RFC = RFC.predict(novo_xv_teste)

    return print("\n\nPrevisão LR: {} \nPrevisão DT: {} \nPrevisão GBC: {} \nPrevisão RFC: {}".format(rotulo_saida(prever_LR[0]), 
                                                                                                              rotulo_saida(prever_DT[0]), 
                                                                                                              rotulo_saida(prever_GBC[0]), 
                                                                                                              rotulo_saida(prever_RFC[0])))

In [ ]:
noticias = str(input())
teste_manual(noticias)